In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

#import keras api needed to implement deep learning techiques
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense, BatchNormalization, Conv2D, MaxPool2D, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator

#Allow charts and graphics to display right below the page of browser setup
%matplotlib inline

# Load image data

We load our collected image data representing five different classes, plastic, metal, glass, and other. Since we are using transfer learning with imagenet, we load it with preprocessing for this network using ImageDataGenerator

In [ ]:
# Remember to mount drive
image_filepath = '/drive/MyDrive/smart_bin_image_data/' # Replace with relevant filepath

train_batch = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.vgg16.preprocess_input,
    shear_range=0.2,
    zoom_range=0.2,
    width_shit_range=0.2,
    height_shift_range=0.2,
    validation_split=0.1).flow_from_directory(
        directory=image_filepath,
        target_size=(500, 500),
        classes=['paper', 'plastic', 'glass', 'metal', 'other'],
        batch_size=64,
    )

# Defining the Architecture

We now define the model that we use to train over the images. Since this is transfer learning, we use imagenet and finetune only the last few layers. Note that we had to adjust the 

In [ ]:
DIMS = 500

# First import in imagenet model
imagenet = tf.keras.applications.VGG16(input_shape=DIMS, 
                                        include_top=False,
                                        weights='imagenet')
imagenet.trainable = False # Make sure not to train this part of the model

# Now we can define our own model on top of this
model = Sequential()
model.add(imagenet)
model.add(Dense(units=128, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(units=64, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Dense(units=5, activation='softmax')) # Final layer with softmax for class probabilities

model.summary()

First, we train the layers we attached to the model

In [ ]:
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(x=train_batches, epochs=10)

Then, we finetune the model for just a few epochs

In [ ]:
imagenet.trainable=True
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(x=train_batches, epochs=3)

In [ ]:
save_path = '/drive/MyDrive/smart_bin_models/imagenet_transfer.hf5'
model.save(save_path)